# Importing Libraries

In [ ]:
%matplotlib notebook
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers, utils
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split

Data was scrapped using scrapping tool

# READING THE DATA 

In [ ]:
matches=pd.read_csv('./data/PSL_2016-2020.csv')
matches.info()

#### NO NULL VALUES FOUND

# DATA

In [ ]:
matches.head()

In [ ]:
matches.tail()

In [ ]:
encode = {'Team1_Code': {'KK':1,'LQ':2,'MS':3,'IU':4,'PZ':5 ,'QG':6},
          'Team2_Code': {'KK':1,'LQ':2,'MS':3,'IU':4,'PZ':5 ,'QG':6},
          'toss_winner_code': {'KK':1,'LQ':2,'MS':3,'IU':4,'PZ':5 ,'QG':6},
          'winner_code': {'KK':1,'LQ':2,'MS':3,'IU':4,'PZ':5 ,'QG':6,'MT':7,'MA':8},
         'toss_decision': {'field':1,'bat':2}}
matches.replace(encode,inplace=True)
matches.head()

In [ ]:
matches.describe()

#### For Finding The Team code the key or value

In [ ]:
dicVal = encode['winner_code']
print(dicVal['QG']) #key value
print(list(dicVal.keys())[list(dicVal.values()).index(6)]) #find key by value search 

In [ ]:
df = pd.DataFrame(matches)
df.describe()

# Saving Total Winnig by each team in DataFrame

In [ ]:
wins = df["winner_code"]
tot_wins = dict(wins.value_counts())
tot_wins

In [ ]:
df['team1_total_wins'] = df['Team1_Code'].map(tot_wins)
df['team2_total_wins'] = df['Team2_Code'].map(tot_wins)
df.head(3)

# Setting The Input Parameters

In [ ]:
matches = matches[['Team1_Code','Team2_Code','toss_winner_code','toss_decision','winner_code',"team1_total_wins","team2_total_wins"]]
matches.head()

# Finding out the Total Number of Wins by each Team

In [ ]:
temp1=df['toss_winner_code'].value_counts(sort=True)
temp2=df['winner_code'].value_counts(sort=True)
#'No of toss winners by each team'
print('No of toss winners by each team')
for idx, val in temp1.iteritems():
    print('{} -> {}'.format(list(dicVal.keys())[list(dicVal.values()).index(idx)],val))
print('No of match winners by each team')
for idx, val in temp2.iteritems():
    print('{} -> {}'.format(list(dicVal.keys())[list(dicVal.values()).index(idx)],val))
print("Where MA : Match Abandoned\n      MT: Match Tied")

# HISTOGRAM OF TEAM WINS

In [ ]:
df['winner_code'].hist(bins=20)

#### Shows the Team labeled 6(i.e Quetta Gladiators) won most of the matches 

# Visualizing the toss wins and match wins

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,4))
ax1 = fig.add_subplot(121)
ax1.set_xlabel('Team')
ax1.set_ylabel('Count of toss wins')
ax1.set_title("toss winners")
temp1.plot(kind='bar')

ax2 = fig.add_subplot(122)
temp2.plot(kind = 'bar')
ax2.set_xlabel('Team')
ax2.set_ylabel('count of matches won')
ax2.set_title("Match winners")

##### Checking If there is any null value in our data

In [ ]:
 df.apply(lambda x: sum(x.isnull()),axis=0) 
    #find the null values in every column

In [ ]:
df.shape

# Building A predictive Model

K-Fold creates multiple random splits of the dataset into training and validation data.
For each such split, the model is fit to the training data,
and predictive accuracy is assessed using the validation data

In [ ]:
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn import metrics

def classification_model(model, data, predictors, outcome):
    model.fit(data[predictors],data[outcome])
    predictions = model.predict(data[predictors])
    accuracy = metrics.accuracy_score(predictions,data[outcome])
    print('Accuracy : %s' % '{0:.3%}'.format(accuracy))
    kf = KFold(n_splits=7,random_state=0)
    error = []
    for train, test in kf.split(data[predictors]):
        train_predictors = (data[predictors].iloc[train,:])
        train_target = data[outcome].iloc[train]
        model.fit(train_predictors, train_target)
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
 
    print('Cross-Validation Score : %s' % '{0:.3%}'.format(np.mean(error)))

    model.fit(data[predictors],data[outcome]) 

###### To avoid any warnings 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=60)
outcome_var = ['winner_code']
predictor_var = ['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']
classification_model(model, df,predictor_var,outcome_var)

In [ ]:
team1='IU'
team2='QG'
toss_winner='QG'
input=[dicVal[team1],dicVal[team2],dicVal[toss_winner],'2','27','30']
input

In [ ]:
temp1=df['toss_winner_code'].value_counts(sort=True)
temp2=df['winner_code'].value_counts(sort=True)
#'No of toss winners by each team'
print('No of toss winners by each team')
for idx, val in temp1.iteritems():
    print('{} -> {}'.format(list(dicVal.keys())[list(dicVal.values()).index(idx)],val))
print('No of match winners by each team')
for idx, val in temp2.iteritems():
    print('{} -> {}'.format(list(dicVal.keys())[list(dicVal.values()).index(idx)],val))
print("Where MA : Match Abandoned\n      MT: Match Tied")

field: 1 bat 2

# MAKING PREDICTIONS

In [ ]:
team1='IU'
team2='QG'
toss_winner='QG'
input=[dicVal[team1],dicVal[team2],dicVal[toss_winner],'2','27','30']
input = np.array(input).reshape((1, -1))
output=model.predict(input)
winner = list(dicVal.keys())[list(dicVal.values()).index(output)] #find key by value search output
print(f"The Winner is {winner}")

In [ ]:
team1='QG'
team2='PZ'
toss_winner='QG'
input=[dicVal[team1],dicVal[team2],dicVal[toss_winner],'1','30','28']

input = np.array(input).reshape((1, -1))
output=model.predict(input)
winner = list(dicVal.keys())[list(dicVal.values()).index(output)] #find key by value search output
print(f"The Winner is {winner}")

In [ ]:
team1='KK'
team2='LQ'
toss_winner='LQ'
input=[dicVal[team1],dicVal[team2],dicVal[toss_winner],'1','18','10']

input = np.array(input).reshape((1, -1))
output=model.predict(input)
winner = list(dicVal.keys())[list(dicVal.values()).index(output)] #find key by value search output
print(f"The Winner is {winner}")

In [ ]:
team1='IU'
team2='KK'
toss_winner='KK'
input=[dicVal[team1],dicVal[team2],dicVal[toss_winner],'1','27','18']

input = np.array(input).reshape((1, -1))
output=model.predict(input)
winner = list(dicVal.keys())[list(dicVal.values()).index(output)] #find key by value search output
print(f"The Winner is {winner}")

#### Important features of model

In [ ]:
imp_input = pd.Series(model.feature_importances_, index=predictor_var).sort_values(ascending=False)
print(imp_input)

# Features Plotting Functions

In [ ]:
def plot_feature_importances(model, feature_names):
    import numpy as np
    c_features = len(feature_names)
    plt.barh(range(c_features), model.feature_importances_)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature name")
    plt.yticks(np.arange(c_features), feature_names)

In [ ]:
imp_input = pd.Series(model.feature_importances_, index=predictor_var).sort_values(ascending=False)
print(imp_input)
plt.figure(figsize=(7,4))
plt.title("Model's Feature Importance")
plot_feature_importances(model, predictor_var)
plt.tight_layout()
plt.show()

# DATA

In [ ]:
y = df[['winner_code']]
X = df[['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# K Nearest Neighbor


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 2)
knn.fit(X_train, y_train)
print('Psl dataset: KNN')
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of KNN classifier on test set: {:.2f}'
     .format(knn.score(X_test, y_test)))

In [ ]:
k_range = range(1,20)
scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    scores.append(knn.score(X_test, y_test))

plt.figure()
plt.title("K value VS accuracy")
plt.xlabel('k')
plt.ylabel('accuracy')
plt.scatter(k_range, scores)
plt.xticks([0,5,10,15,20]);

In [ ]:
t = [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]

knn = KNeighborsClassifier(n_neighbors = 1)

plt.figure()

for s in t:

    scores = []
    for i in range(1,1000):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1-s)
        knn.fit(X_train, y_train)
        scores.append(knn.score(X_test, y_test))
    plt.plot(s, np.mean(scores), 'bo')
plt.title("Training set proportion VS Accuracy")
plt.xlabel('Training set proportion (%)')
plt.ylabel('accuracy');

# Using Decision Tree

In [ ]:
#Data
y = df[['winner_code']]
X = df[['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(random_state=0,max_depth=5, max_leaf_nodes=5,min_samples_leaf=5)
DT.fit(X_train, y_train)
print('Psl dataset: DT')
print('Accuracy of DT classifier on training set: {:.2f}'
     .format(DT.score(X_train, y_train)))
print('Accuracy of DT classifier on test set: {:.2f}'
     .format(DT.score(X_test, y_test)))

In [ ]:
leaf_nodes_range = range(2,6)
scores = []

for x in leaf_nodes_range:
    DT = DecisionTreeClassifier(random_state=0,max_depth=5, max_leaf_nodes=x,min_samples_leaf=5)
    DT.fit(X_train, y_train)
    scores.append(DT.score(X_test, y_test))

plt.figure()
plt.title("Leaf Nodes VS Accuracy")
plt.xlabel('nodes')
plt.ylabel('accuracy')
plt.scatter(leaf_nodes_range, scores)
plt.xticks([1,2,3,4,5]);

In [ ]:
depth_range = range(1,6)
scores = []

for x in depth_range:
    DT = DecisionTreeClassifier(random_state=0,max_depth=x, max_leaf_nodes=13,min_samples_leaf=5)
    DT.fit(X_train, y_train)
    scores.append(DT.score(X_test, y_test))

plt.figure()
plt.title("Depth VS Accuracy")
plt.xlabel('depth')
plt.ylabel('accuracy')
plt.scatter(depth_range, scores)
plt.xticks([0,1,2,3,4,5]);

In [ ]:
t = [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]
plt.figure()
for s in t:
    scores = []
    for i in range(1,1000):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1-s)
        DT.fit(X_train, y_train)
        scores.append(DT.score(X_test, y_test))
    plt.plot(s, np.mean(scores), 'bo')
plt.title("Training set proportion VS Accuracy")
plt.xlabel('Training set proportion (%)')
plt.ylabel('accuracy');

# Using Random Forest

In [ ]:
#Data
y = df[['winner_code']]
X = df[['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(random_state=0,max_depth=4)
RF.fit(X_train, y_train)
print('Psl dataset:RF')
print('Accuracy of RF classifier on training set: {:.2f}'
     .format(RF.score(X_train, y_train)))
print('Accuracy of RF classifier on test set: {:.2f}'
     .format(RF.score(X_test, y_test)))

In [ ]:
depth_range = range(1,6)
scores = []

for z in depth_range:
    RF = RandomForestClassifier(random_state=0,max_depth=z)
    RF.fit(X_train, y_train)
    scores.append(RF.score(X_test, y_test))

plt.figure()
plt.title("Depth VS Accuracy")
plt.xlabel('depth')
plt.ylabel('accuracy')
plt.scatter(depth_range, scores)
plt.xticks([0,1,2,3,4,5]);

In [ ]:
t = [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]
plt.figure()
for s in t:
    scores = []
    for i in range(1,1000):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1-s)
        DT.fit(X_train, y_train)
        scores.append(DT.score(X_test, y_test))
    plt.plot(s, np.mean(scores), 'bo')
plt.title("Training set proportion VS Accuracy")
plt.xlabel('Training set proportion (%)')
plt.ylabel('accuracy');

# Using KFold Validation

In [ ]:
#Data
y = df[['winner_code']]
X = df[['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn import metrics

def classification_model(model, data, predictors, outcome):
    model.fit(data[predictors],data[outcome])
    predictions = model.predict(data[predictors])
    accuracy = metrics.accuracy_score(predictions,data[outcome])
    print('Accuracy : %s' % '{0:.3%}'.format(accuracy))
    kf = KFold(n_splits=7,random_state=None)
    error = []
    for train, test in kf.split(data[predictors]):
        train_predictors = (data[predictors].iloc[train,:])
        train_target = data[outcome].iloc[train]
        model.fit(train_predictors, train_target)
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
 
    print('Cross-Validation Score : %s' % '{0:.3%}'.format(np.mean(error)))

    model.fit(data[predictors],data[outcome]) 

# KNN with cross validation

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors = 1)
outcome_var = ['winner_code']
predictor_var = ['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']
classification_model(model, df,predictor_var,outcome_var)

# Decision Tree with cross Validation

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=0)
outcome_var = ['winner_code']
predictor_var = ['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']
classification_model(model, df,predictor_var,outcome_var)

# Random Forest with Cross Validation

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=60)
outcome_var = ['winner_code']
predictor_var = ['Team1_Code','Team2_Code','toss_winner_code','toss_decision','team1_total_wins','team2_total_wins']
classification_model(model, df,predictor_var,outcome_var)